# Collect all Paths 
This does not need to be ran anymore we have the list in paths.txt now. Just read that into simulation_paths and you are good to go with collecting data

In [1]:
import s3fs
import zarr
import json
from tqdm import tqdm
import pandas as pd
import pickle
from pprint import pprint
import os
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
endpoint = 'https://wifire-data.sdsc.edu:9000'
access_key = os.getenv("ACCESS_KEY")
secret_key = os.getenv("SECRET_KEY")

fs = s3fs.S3FileSystem(key=access_key,
    secret=secret_key,
    client_kwargs={
        'endpoint_url': endpoint,
        'verify': False
    },
    skip_instance_cache=False
)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

name = 'quicfire.zarr'
bucket = 'burnpro3d/d'

root = list(fs.ls(bucket))

simulation_paths = []
print("successfully authenticated")

successfully authenticated


In [13]:
def get_paths(rangestart,rangeend):
    global fs, bucket
    paths = fs.ls(bucket)
    for path in paths[rangestart:rangeend]:
        paths = fs.ls(path)
        for path in paths:
            get_child_directories(path)

def get_child_directories(path):
    global fs
    global simulation_paths
    paths = fs.ls(path)
    for p in paths:
        if "run_" in p:
            simulation_paths.append(p)

def read_paths():
    paths = []
    with open("paths.txt","r") as f:
        paths = f.read().splitlines() 
    return paths


In [14]:
# with open("paths.txt", "a") as file:
#     for i in simulation_paths:
#         file.write(i)
#         file.write("\n")

In [17]:
simulation_paths = read_paths()
len(simulation_paths)

26997

# Now Extract Data

In [18]:
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS

bucket = "devin"
url= "http://localhost:8086"
token = os.getenv("INFLUXDB_TOKEN")
org = "SDSC internship"


client = influxdb_client.InfluxDBClient(
    url=url,
    token=token,
    org=org
)

# Write script
write_api = client.write_api(write_options=SYNCHRONOUS)

p = influxdb_client.Point("hi").tag("location", "Czech").field("height",6).field("age",8)
write_api.write(bucket=bucket, org=org, record=p)

In [19]:
json_payload = []
data = {
    "measurement": "simulation_run",
    "tags": {
        "wifire"
    },
    "time": "2022-10-22T12:52:03.903821-07:00",
    "fields": {
        # data goes here
    }
}
# json_payload.append(data)
# client.write_points(json_payload)

In [38]:
KEEP_ATTRIBUTES = {
    'path': lambda d: None,
    'canopy_moisture': lambda d: d['canopy_moisture'],
    # 'dz',
    'extent': lambda d: d['extent'],
    'extent_fmt': lambda d: d['extent_fmt'],
    'fire_grid': lambda d: d['fire_grid'],
    'fuel': lambda d: d['fuel'],
    'ignition': lambda d: d['ignition'],
    'output': lambda d: d['output'],
    'resolution': lambda d: d['resolution'],
    'resolution_units': lambda d: d['resolution_units'],
    'run_binary': lambda d: d['run_binary'],
    'run_end': lambda d: d['run_end'],
    'run_max_mem_rss_bytes': lambda d: d['run_max_mem_rss_bytes'],
    'run_start': lambda d: d['run_start'],
    'seed': lambda d: d['seed'],
    'sim_time': lambda d: d['sim_time'],
    'surface_moisture': lambda d: d['surface_moisture'],
    'threads': lambda d: d['threads'],
    'timestep': lambda d: d['timestep'],
    'topo': lambda d: d['topo'],
    'wind_direction': lambda d: d['wind_direction'],
    'wind_speed': lambda d: d['wind_speed']
}
filenotfound = []

def get_df(count):
    global simulation_paths, KEEP_ATTRIBUTES, incomplete, filenotfound

    df = pd.DataFrame([], columns=KEEP_ATTRIBUTES.keys())

    i = 0
    for p in tqdm(simulation_paths[:count]):
        try:
            with fs.open(p + '/' + name + '/.zattrs') as f:
                data=json.load(f)
        except:
            filenotfound.append(p)
            print("FileNotFound error on path {",p,"}")
            continue
            
        row = []
        for k,expr in KEEP_ATTRIBUTES.items():
            try:
                value = KEEP_ATTRIBUTES[k](data)
            except KeyError:
                value = None
            row.append(value)
        row[0] = p
        df.loc[i] = row
        i+=1

    return df

pd.set_option('display.max_columns', None)
print(filenotfound)

[]


In [126]:
simulation_paths = read_paths()
df = get_df(20)

 85%|██████████████████████████████████      | 17/20 [00:06<00:00,  5.65it/s]

FileNotFound error on path { burnpro3d/d/00/20/run_0020b844-1233-4547-a9e2-5c9a11785939 }


100%|████████████████████████████████████████| 20/20 [00:10<00:00,  1.90it/s]


In [127]:
# print(df["sim_time"])
# df.dropna(subset=['name', 'toy'])

df2 = df[['sim_time', 'run_binary', 'run_end', 'run_start', 'wind_direction','extent']]
print(df2.head())
df2 = df2.rename(columns={"run_end": "stop", "run_start": "start"})
df2

   sim_time                        run_binary  \
0      3200        /quicfire/quicfire_Jan2022   
1      5983  /quicfire/quicfire-v5.2-ucsd-rel   
2      2100        /quicfire/quicfire_Jan2022   
3     18600         /quicfire/quicfire-v4.0.1   
4       600         /quicfire/quicfire-v5.1.1   

                            run_end                         run_start  \
0  2022-04-07T20:41:23.473740-07:00  2022-04-07T15:28:35.045121-07:00   
1                              None  2023-07-18T14:17:12.279765-07:00   
2  2022-03-31T17:42:42.149814-07:00  2022-03-31T16:13:27.935085-07:00   
3  2022-10-22T19:05:52.183442-07:00  2022-10-22T12:52:03.903821-07:00   
4  2023-01-09T18:41:19.260942-08:00  2023-01-09T18:33:02.835216-08:00   

   wind_direction                                             extent  
0           225.0  [-1019.635396216905, 587.5323697931051, 780.36...  
1           150.0       [-939420.0, 1474018.0, -938616.0, 1473212.0]  
2           180.0             [-2048049, 1866939, -20

,sim_time,run_binary,stop,start,wind_direction,extent
0,3200,/quicfire/quicfire_Jan2022,2022-04-07T20:41:23.473740-07:00,2022-04-07T15:28:35.045121-07:00,225.000000,"[-1019.635396216905, 587.5323697931051, 780.36..."
1,5983,/quicfire/quicfire-v5.2-ucsd-rel,None,2023-07-18T14:17:12.279765-07:00,150.000000,"[-939420.0, 1474018.0, -938616.0, 1473212.0]"
2,2100,/quicfire/quicfire_Jan2022,2022-03-31T17:42:42.149814-07:00,2022-03-31T16:13:27.935085-07:00,180.000000,"[-2048049, 1866939, -2046749, 1866139]"
3,18600,/quicfire/quicfire-v4.0.1,2022-10-22T19:05:52.183442-07:00,2022-10-22T12:52:03.903821-07:00,245.000000,"[939794.0, 1235562.0, 941860.0, 1233570.0]"
4,600,/quicfire/quicfire-v5.1.1,2023-01-09T18:41:19.260942-08:00,2023-01-09T18:33:02.835216-08:00,270.000000,"[0, 600, 600, 0]"
5,15963,/quicfire/quicfire-v5.1.1,2023-02-01T10:46:25.591971-08:00,2023-02-01T10:33:03.706186-08:00,335.000000,"[1335078.0, 1098858.0, 1336356.0, 1096868.0]"
6,100,None,None,None,NaN,"[-1956516, 1290750, -1955516, 1289750]"
7,1800,/quicfire/quicfire_Jan2022,None,2022-05-18T16:38:15.105115-07:00,15.000000,"[-496.61133, 589.10388, 303.38867, -510.89612]"
8,600,/quicfire/quicfire-v5.1.1,2023-02-24T11:37:45.285185-08:00,2023-02-24T10:56:05.393579-08:00,300.000000,"[0, 600, 600, 0]"
9,1000,/quicfire/quicfire_Jan2022,2022-07-13T01:27:31.027213-07:00,2022-07-13T01:18:55.686756-07:00,284.422885,"[1120178.154045004, 913143.8979546748, 1120778..."


In [68]:
df2.dropna()

,sim_time,run_binary,run_end,run_start,wind_direction,extent
0,3200,/quicfire/quicfire_Jan2022,2022-04-07T20:41:23.473740-07:00,2022-04-07T15:28:35.045121-07:00,225.000000,"[-1019.635396216905, 587.5323697931051, 780.36..."
2,2100,/quicfire/quicfire_Jan2022,2022-03-31T17:42:42.149814-07:00,2022-03-31T16:13:27.935085-07:00,180.000000,"[-2048049, 1866939, -2046749, 1866139]"
3,18600,/quicfire/quicfire-v4.0.1,2022-10-22T19:05:52.183442-07:00,2022-10-22T12:52:03.903821-07:00,245.000000,"[939794.0, 1235562.0, 941860.0, 1233570.0]"
4,600,/quicfire/quicfire-v5.1.1,2023-01-09T18:41:19.260942-08:00,2023-01-09T18:33:02.835216-08:00,270.000000,"[0, 600, 600, 0]"
5,15963,/quicfire/quicfire-v5.1.1,2023-02-01T10:46:25.591971-08:00,2023-02-01T10:33:03.706186-08:00,335.000000,"[1335078.0, 1098858.0, 1336356.0, 1096868.0]"
8,600,/quicfire/quicfire-v5.1.1,2023-02-24T11:37:45.285185-08:00,2023-02-24T10:56:05.393579-08:00,300.000000,"[0, 600, 600, 0]"
9,1000,/quicfire/quicfire_Jan2022,2022-07-13T01:27:31.027213-07:00,2022-07-13T01:18:55.686756-07:00,284.422885,"[1120178.154045004, 913143.8979546748, 1120778..."
10,3200,/quicfire/quicfire_Jan2022,2022-04-08T11:28:41.298723-07:00,2022-04-08T08:05:05.423825-07:00,225.000000,"[-1019.635396216905, 587.5323697931051, 780.36..."
11,1000,/quicfire/quicfire_Jan2022,2022-07-12T23:22:44.224985-07:00,2022-07-12T23:03:26.779229-07:00,267.353334,"[1120178.154045004, 913143.8979546748, 1120778..."
12,2100,/quicfire/quicfire_Jan2022,2022-03-31T15:07:08.550983-07:00,2022-03-31T13:30:40.037256-07:00,225.000000,"[-2048049, 1866939, -2046749, 1866139]"


In [121]:
data = {"height":[42, 13], "age": [4.24,31]}
df = pd.DataFrame(data)
df.rename(columns={"age":"asdf","height":"he"})
print(df)

df = pd.DataFrame({"A": [1, 2, 3], "B": [4, 5, 6]})
df = df.rename(columns={"A": "alpha", "B": "cockroach"})
df

   height    age
0      42   4.24
1      13  31.00


,alpha,cockroach
0,1,4
1,2,5
2,3,6


In [ ]:
from influxdb_client import InfluxDBClient, Point, WriteOptions
from influxdb_client.client.write_api import SYNCHRONOUS
# Preparing Dataframe: 
system_stats.drop(columns=['result', 'table','start','stop'])
# DataFrame must have the timestamp column as an index for the client. 
system_stats.set_index("_time")
_write_client.write(bucket.name, record=system_stats, data_frame_measurement_name='cpu',
                    data_frame_tag_columns=['cpu'])
Copy


# Now Query for the Data we entered

In [104]:
query_api = client.query_api()
query = 'from(bucket:"devin")\
|> range(start: -10m)\
|> filter(fn:(r) => r._measurement == "simulation_run")'
result = query_api.query(org=org, query=query)

In [110]:
client.default_tags

In [98]:
results = []
for table in result:
  for record in table.records:
    results.append((record.get_field(), record.get_value()))

print(results)
# [(temperature, 25.3)]

[('age', 67), ('age', 90), ('age', 8), ('height', 42), ('height', 231), ('height', 231), ('height', 231), ('height', 78), ('height', 6), ('temperature', 29.3), ('temperature', 29.3), ('temperature', 29.3), ('temperature', 29.3), ('temperature', 8.9), ('temperature', 89.9)]
